In [1]:
# 1. 임의의 입력 생성하기

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

train_X = [[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]
print(np.shape(train_X))

(4, 5)


In [2]:
# 위 입력은 단어 벡터의 차원은 5이고, 문장의 길이가 4인 경우를 가정한 입력입니다.
# 다시 말해 4번의 시점(timesteps)이 존재하고, 각 시점마다 5차원의 단어 벡터가 입력으로 사용됩니다.
# 그러나, RNN은 2D 텐서가 아니라 3D 텐서를 입력을 받습니다.
# 즉, 위에서 만든 2D 텐서를 3D 텐서로 변경합니다. 이는 배치 크기 1을 추가해주므로서 해결합니다.

train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape)

(1, 4, 5)


In [3]:
# (batch_size, timesteps, input_dim)에 해당되는 (1, 4, 5)의 크기를 가지는 3D 텐서가 생성되었습니다.
# batch_size는 한 번에 RNN이 학습하는 데이터의 양을 의미하지만, 여기서는 샘플이 1개 밖에 없으므로 batch_size는 1입니다.

In [8]:
# 2. Simple RNN 이해하기
# 위에서 생성한 데이터를 SimpleRNN의 입력으로 사용하여 SimpleRNN의 출력값을 이해해보겠습니다.
# SimpleRNN에는 여러 인자가 있으며 대표적인 인자로 return_sequences와 return_state가 있습니다.
# 기본값으로는 둘 다 False로 지정되어져 있으므로 별도 지정을 하지 않을 경우에는 False로 처리됩니다.
# 우선, 은닉 상태의 크기를 3으로 지정하고, 두 인자 값이 모두 False일 때의 출력값을 보겠습니다.
# 앞으로의 실습에서 SimpleRNN을 매번 재선언하므로 은닉 상태의 값 자체는 매번 초기화되어 이전 출력과 값의 일관성은 없습니다.
# 출력값 자체보다는 해당 값의 크기(shape)에 주목해야합니다.

In [4]:
rnn = SimpleRNN(3)
# rnn = SimpleRNN(3, return_sequences=False, return_state=False)와 동일.
hidden_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))

hidden state : [[ 0.8916486   0.79469365 -0.9893858 ]], shape: (1, 3)


In [6]:
# (1, 3) 크기의 텐서가 출력되는데, 이는 마지막 시점의 은닉 상태입니다. 은닉 상태의 크기를 3으로 지정했음을 주목합시다.
# 기본적으로 return_sequences가 False인 경우에는 SimpleRNN은 마지막 시점의 은닉 상태만 출력합니다.
# 이번에는 return_sequences를 True로 지정하여 모든 시점의 은닉 상태를 출력해봅시다.

rnn = SimpleRNN(3, return_sequences=True)
hidden_states = rnn(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))

hidden states : [[[ 0.9941313   0.7198722  -0.9987676 ]
  [ 0.86420375  0.97608244  0.50694644]
  [ 0.8843333   0.39389762  0.7669031 ]
  [-0.54434085  0.31887457  0.8800741 ]]], shape: (1, 4, 3)


In [ ]:
# (1, 4, 3) 크기의 텐서가 출력됩니다. 앞서 입력 데이터는 (1, 4, 5)의 크기를 가지는 3D 텐서였고,
# 그 중 4가 시점(timesteps)에 해당하는 값이므로 모든 시점에 대해서 은닉 상태의 값을 출력하여 (1, 4, 3) 크기의 텐서를 출력하는 것입니다.

# return_state가 True일 경우에는 return_sequences의 True/False 여부와 상관없이 마지막 시점의 은닉 상태를 출력합니다.
# 가령, return_sequences가 True이면서, return_state를 True로 할 경우 SimpleRNN은 두 개의 출력을 리턴합니다.

In [7]:
rnn = SimpleRNN(3, return_sequences=True, return_state=True)
hidden_states, last_state = rnn(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))

hidden states : [[[ 0.8935383  -0.89594173 -0.5480382 ]
  [ 0.2789381  -0.5591895   0.06882747]
  [-0.20488802 -0.95139366  0.84292525]
  [-0.37173617  0.4526819  -0.95848864]]], shape: (1, 4, 3)
last hidden state : [[-0.37173617  0.4526819  -0.95848864]], shape: (1, 3)


In [10]:
# 첫번째 출력은 return_sequences=True로 인한 출력으로 모든 시점의 은닉 상태입니다.
# 두번째 출력은 return_state=True로 인한 출력으로 마지막 시점의 은닉 상태입니다.
# 실제로 출력을 보면 모든 시점의 은닉 상태인 (1, 4, 3) 텐서의 마지막 벡터값이 return_state=True로 인해 출력된 벡터값과 일치하는 것을 볼 수 있습니다.
# 그렇다면 return_sequences는 False인데, retun_state가 True인 경우는 어떨까요?

rnn = SimpleRNN(3, return_sequences=False, return_state=True)
hidden_state, last_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))

hidden state : [[ 0.9697794  -0.8525398   0.96484786]], shape: (1, 3)
last hidden state : [[ 0.9697794  -0.8525398   0.96484786]], shape: (1, 3)


In [11]:
# 두 개의 출력 모두 마지막 시점의 은닉 상태를 출력하게 됩니다.

In [ ]:
# 3. LSTM 이해하기
# 실제로 SimpleRNN이 사용되는 경우는 거의 없습니다.
# 이보다는 LSTM이나 GRU을 주로 사용하는데, 이번에는 임의의 입력에 대해서 LSTM을 사용할 경우를 보겠습니다.
# 우선 return_sequences를 False로 두고, return_state가 True인 경우를 봅시다.

In [13]:
lstm = LSTM(3, return_sequences=False, return_state=True)
hidden_state, last_state, last_cell_state = lstm(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden state : [[0.26478314 0.23940189 0.0804053 ]], shape: (1, 3)
last hidden state : [[0.26478314 0.23940189 0.0804053 ]], shape: (1, 3)
last cell state : [[1.5838846  0.51239425 0.11952263]], shape: (1, 3)


In [ ]:
# 이번에는 SimpleRNN 때와는 달리, 세 개의 결과를 반환합니다.
# return_sequences가 False이므로 우선 첫번째 결과는 마지막 시점의 은닉 상태입니다.
# 그런데 LSTM이 SimpleRNN과 다른 점은 return_state를 True로 둔 경우에는 마지막 시점의 은닉 상태뿐만 아니라 셀 상태까지 반환한다는 점입니다.
# 이번에는 return_sequences를 True로 바꿔보겠습니다.

In [14]:
lstm = LSTM(3, return_sequences=True, return_state=True)
hidden_states, last_hidden_state, last_cell_state = lstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state : {}, shape: {}'.format(last_hidden_state, last_hidden_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden states : [[[ 0.00930385  0.00874694  0.2833836 ]
  [ 0.06245268  0.01182099  0.5062674 ]
  [ 0.11605892  0.177935    0.5634438 ]
  [-0.08814912 -0.02936115  0.64714605]]], shape: (1, 4, 3)
last hidden state : [[-0.08814912 -0.02936115  0.64714605]], shape: (1, 3)
last cell state : [[-0.11569324 -0.07186611  1.2547723 ]], shape: (1, 3)


In [ ]:
# return_state가 True이므로 두번째 출력값이 마지막 은닉 상태, 세번째 출력값이 마지막 셀 상태인 것은 변함없지만
# return_sequences가 True이므로
# 첫번째 출력값은 모든 시점의 은닉 상태가 출력됩니다.

In [17]:
# 4. Bidirectional(LSTM) 이해하기
# 난이도를 조금 올려서 양방향 LSTM의 출력값을 확인해보겠습니다.
# return_sequences가 True인 경우와 False인 경우에 대해서 은닉 상태의 값이
# 어떻게 바뀌는지 직접 비교하기 위해서 이번에는 출력되는 은닉 상태의 값을 고정시켜주겠습니다.

In [18]:
k_init = tf.keras.initializers.Constant(value=0.1)
b_init = tf.keras.initializers.Constant(value=0)
r_init = tf.keras.initializers.Constant(value=0.1)

In [ ]:
# 우선 return_sequences가 False이고, return_state가 True인 경우입니다.

In [20]:
bilstm = Bidirectional(LSTM(3, return_sequences=False, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[0.6303138 0.6303138 0.6303138 0.7038734 0.7038734 0.7038734]], shape: (1, 6)
forward state : [[0.6303138 0.6303138 0.6303138]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


In [ ]:
# 이번에는 무려 6개의 값을 반환합니다.
# return_state가 True인 경우에는 정방향 LSTM의 은닉 상태와 셀 상태, 역방향 LSTM의 은닉 상태와 셀 상태 4가지를 반환하기 때문입니다.
# 첫번째 출력값의 크기가 (1, 6)인 것에 주목합시다.
# 이는 return_sequences가 False인 경우 정방향 LSTM의 마지막 시점의 은닉 상태와 역방향 LSTM의 첫번째 시점의 은닉 상태가 연결된 채 반환되기 때문입니다.
# 그림으로 표현하면 아래와 같이 연결되어 다음층에서 사용됩니다.

In [ ]:
# 정방향 LSTM의 마지막 시점의 은닉 상태값과 역방향 LSTM의 첫번째 은닉 상태값을 기억해둡시다.
# 현재 은닉 상태의 값을 고정시켜두었기 때문에 return_sequences를 True로 할 경우, 출력이 어떻게 바뀌는지 비교가 가능합니다.

In [26]:
bilstm = Bidirectional(LSTM(3, return_sequences=True, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))


hidden states : [[[0.35906473 0.35906473 0.35906473 0.7038734  0.7038734  0.7038734 ]
  [0.55111325 0.55111325 0.55111325 0.58863586 0.58863586 0.58863586]
  [0.59115744 0.59115744 0.59115744 0.3951699  0.3951699  0.3951699 ]
  [0.6303138  0.6303138  0.6303138  0.21942244 0.21942244 0.21942244]]], shape: (1, 4, 6)
forward state : [[0.6303138 0.6303138 0.6303138]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


In [ ]:
# hidden states의 출력값에서는 이제 모든 시점의 은닉 상태가 출력됩니다.
# 역방향 LSTM의 첫번째 시점의 은닉 상태는 더 이상 정방향 LSTM의 마지막 시점의 은닉 상태와 연결되는 것이 아니라
# 정방향 LSTM의 첫번째 시점의 은닉 상태와 연결됩니다.
# 그림으로 표현하면 다음과 같이 연결되어 다음층의 입력으로 사용됩니다.